In [ ]:
import pandas as pd

In [ ]:
all_qac=pd.read_csv('all_qac_triplets.csv').drop_duplicates(subset=['original_ques','context','filename'])[['original_ques','context','filename']]
#all_qac=pd.read_csv('all_qac_triplets.csv')
all_qac.head(5)

In [ ]:
contexts = all_qac.drop_duplicates(subset=['context'])
unique_context = contexts["context"].tolist()


In [ ]:
unique_context[0:2]


In [ ]:
import nltk
def break_into_sentences(passage):
    sentences = nltk.sent_tokenize(passage)
    return sentences

In [ ]:
break_into_sentences(unique_context[3363])

In [ ]:
print(len(unique_context))

In [ ]:
from langdetect import detect
from textblob import TextBlob
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False



In [ ]:
def at_least_two_sensible_sentences(context):
    sentences = nltk.sent_tokenize(context)
    english_sentences = [sentence for sentence in sentences if is_english(sentence)]
    return len(english_sentences) >= 2

In [ ]:
#selecting random sentances from contexts
# import random
# def get_english_contexts(unique_context, target_count=1000):
#     context_length = len(unique_context)
#     selected_context = []
#     random_numbers=[]
#     while len(selected_context) < target_count:
#         random_index = random.randint(0, context_length - 1)
#         #print(at_least_two_sensible_sentences(unique_context[random_index]))
#         if at_least_two_sensible_sentences(unique_context[random_index]): # type: ignore
#             if unique_context[random_index] not in selected_context:
#                 selected_context.append(unique_context[random_index])
#                 random_numbers.append(random_index)

#     return selected_context,random_numbers

# selected_context,random_numbers = get_english_contexts(unique_context)

# # Display the first two selected contexts
# selected_context[0:2]

In [ ]:
# import pickle
# with open('selected_context.pkl','wb') as f:
#     pickle.dump(random_numbers,f)

In [ ]:
import pickle
with open('selected_context.pkl','rb') as f:
    random_numbers=pickle.load(f)
#random_numbers    
selected_context = [unique_context[i] for i in random_numbers]

In [ ]:

def is_english_and_sensible(sentence):
    try:
        # Detect language
        if detect(sentence) != 'en':
            return False
        
        # Use TextBlob to check if the sentence makes sense (basic check)
        blob = TextBlob(sentence)
        if len(blob.words) < 3 or not blob.correct():
            return False
        return True
    except:
        return False

In [ ]:
#select sentances 
# import random
# sentance_number={}
# sentances=[]
# for i in range(len(selected_context)):
#     if(i%100==0):
#      print(i)
#     context=selected_context[i]
#     sentances=break_into_sentences(context)
#     random_num=0
#     is_valid=False
#     while(not is_valid):
#         random_num = random.randrange(0, len(sentances))
#         sentence=sentances[random_num]
#         is_valid=is_english_and_sensible(sentence)
#         #print(is_valid)
#     sentance_number[i]=random_num
#     sentances.append(sentence)
    
# sentances    

In [ ]:
# import json
# with open('selected_sentence.json','w') as f:
#     json.dump(sentance_number,f)

In [ ]:
import json
with open('selected_sentence.json','r') as f:
    sentance_number=json.load(f)

In [ ]:
sentance_number

In [ ]:

selected_sentances=[]
for i in range(len(selected_context)):
    if(i%100==0):
     print(i)
    context=selected_context[i]
    sentance=break_into_sentences(context)[sentance_number[str(i)]]
    
    
    selected_sentances.append(sentance)
    
selected_sentances   

In [ ]:
print(len(selected_sentances)) # type: ignore

In [ ]:
#prompt to rephrase the text
def prompt1(alltext):
    return f'''You will be provided with text snippets extracted from a clinical trial protocol document. Your task is to generate up to two rephrased versions for each text snippet. Below are the text snippets:

<text>

{alltext}

</text>

Please generate the rephrased versions of the text snippets. Only rephrase the text given within the tags.

Each original text snippet and its rephrased versions should follow this format:
T1: Original text.
T1.1: First rephrased version.
T1.2: Second rephrased version.

To create effective paraphrases, avoid using the same words as in the original text. For example, if the original text is 'What is the patient's age?' a good paraphrase could be 'How old is the patient?' or 'What is the age of the patient?'.

The overall format of your response should match what is shown between the tags. Ensure to strictly follow the formatting and spacing.

<example>

T1: Patients were assessed for adverse events.
T1.1: Patients were evaluated for side effects.
T1.2: Adverse events in patients were monitored.

T2: The primary endpoint was overall survival.
T2.1: The main outcome measure was overall survival.
T2.2: Overall survival was the primary endpoint.

T3: Informed consent was obtained from all participants.
T3.1: All participants provided informed consent.
T3.2: Consent was obtained from all participants in an informed manner.

</example>

Please provide up to two rephrased versions for each text snippet. Ensure that the output includes only the original text and their rephrased versions, formatted as shown in the example. The rephrased text should maintain the same meaning as the original text. Make sure the original text snippet is present in the output and follows the specified format.'''


In [ ]:
def system_prompt(prompt):
    #https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/
    return f'''<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant. Your carefully follow instructions.<|eot_id|><|start_header_id|>user<|end_header_id|>

{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'''

In [ ]:
j=0
prompts=[]
while(j!=len(selected_sentances)):
    text=''
    for i in range(10):
        modified_sentence = selected_sentances[i+j].replace('\n', ' ')
        text=text+f"Q{i+1}:{modified_sentence}\n"
    prompt=system_prompt(prompt1(text))
    prompts.append(prompt) 
    j=j+10  
print(len(prompts))      


In [ ]:
from vllm import LLM, SamplingParams
llm = LLM(model="/lockbox/models/Meta-Llama-3-8B-Instruct", gpu_memory_utilization = 0.8,enforce_eager=True)
sampling_params = SamplingParams(temperature=0.05, top_p=0.95, top_k = 40, max_tokens = 4816,)
out=[]
try:
    generated_outputs = llm.generate(prompts, sampling_params)
    for output in generated_outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        out.append(generated_text) 
except Exception:
            print(Exception)  


In [ ]:
# for i in range(len(out)):
#     temp=out[i]
#     filename=f'./Train_data/all_rephrased_text/out{i}.txt'
#     with open(filename,'w') as f:
#         f.write(temp)

In [ ]:
import re
def extract_rephrased_questions(file_path):
    # Read the text file
    with open(file_path, 'r') as file:
        file_content = file.read()

    # Define the pattern to match the rephrased questions
    pattern = r'^Q\d+\.\d+: (.+)$'


    # Find all matches using re.findall
    rephrased_questions = re.findall(pattern, file_content, re.MULTILINE)

    return rephrased_questions

rephrased_questions = extract_rephrased_questions('./Train_data/all_rephrased_text/out98.txt')
rephrased_questions

In [ ]:
import os
import random

path='./Train_data/all_rephrased_text/'
rephrased=[]
for j in range(100):
    file=f"out{j}.txt"
    full_path=path+file
    rephrased_questions = extract_rephrased_questions(full_path)
    if(len(rephrased_questions)!=20):
        print(full_path)
    for i in range(0,len(rephrased_questions),2):
            num=random.randint(0, 1)
            #print(num)
            if(num==0):rephrased.append(rephrased_questions[i])
            elif(num==1):rephrased.append(rephrased_questions[i+1])
    

print(len(rephrased))

In [ ]:
def at_least_one_sensible_sentences(context):
    sentences = nltk.sent_tokenize(context)
    english_sentences = [sentence for sentence in sentences if is_english(sentence)]
    return len(english_sentences) >= 1

In [ ]:
#select a random context from the same or a different context
import random
from tqdm import tqdm

seen = set()
negatives = []

for i, context in tqdm(enumerate(selected_context), total=len(selected_context), desc="Processing contexts"):
    # if i % 100 == 0:
    #     print(i)

    if random.randint(0, 1) == 0:
        while True:
            context_num = random.randint(0, len(unique_context) - 1)
            not_found=True

            if (context_num not in random_numbers 
                and context_num not in seen 
                and at_least_one_sensible_sentences(unique_context[context_num])):
                
                sentences = break_into_sentences(unique_context[context_num])
                seen.add(context_num)
                
                for sent in sentences:
                    if is_english_and_sensible(sent):
                        negatives.append(sent)
                        not_found=False
                        break
                if(not_found):
                    for sent in sentences:
                        if is_english(sent):
                            negatives.append(sent)
                            not_found=False
                            break
                break    
    else:
        present_sentence = sentance_number[str(i)]
        sentences = break_into_sentences(context)
        not_found=True
        
        for j, sent in enumerate(sentences):
            if j != present_sentence and is_english_and_sensible(sent) and abs(j - present_sentence) > 2:
                negatives.append(sent)
                not_found=False
                break
        if(not_found):
            for j, sent in enumerate(sentences):
                if j != present_sentence and is_english(sent):
                    negatives.append(sent)
                    not_found=False
                    break
                
                
            

        
         

In [ ]:
len(negatives) 

In [ ]:
length=len(negatives) 
negatives_cleaned=[]

for i in range(length):
    neg=negatives[i]
    sentances=break_into_sentences(neg)
    if(len(sentances)==1):
        negatives_cleaned.append(neg)

len(negatives_cleaned)          

In [ ]:
FineTune_dataset=[]
for i in range(len(selected_context)):
    temp={}
    temp['anchor']=selected_sentances[i]
    temp['positive']=rephrased[i]
    temp['negative']=negatives[i]
    FineTune_dataset.append(temp)


FineTune_dataset

In [ ]:
#save the data

In [ ]:
import pickle
with open('anchor.pkl','wb') as f:
    pickle.dump(selected_sentances,f)

In [ ]:
with open('positives.pkl','wb') as f:
    pickle.dump(rephrased,f)

In [ ]:
with open('negatives.pkl','wb') as f:
    pickle.dump(negatives,f)

In [ ]:
import json
with open('FineTuneDataset.json','w') as f:
    json.dump(FineTune_dataset,f)


In [ ]:
def clean_text(path):
    with open(path,'rb') as f:
        li=pickle.load(f)

    filter_text=[text.strip().replace('\n',' ') for text in li]
    with open(path,'wb') as f:
        pickle.dump(filter_text,f)

clean_text('anchor.pkl')
clean_text('positives.pkl')
clean_text('negatives.pkl')